ollamaの起動

In [ ]:
!pip install -q pypdf
!pip install -q pypdf nbformat

In [ ]:
# 1. 依存ツールのインストール
!apt-get update && apt-get install -y zstd pciutils

# 2. Ollamaのインストール
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess
import time
import requests

# 共通設定（ここでモデル名を一括管理します）
# 使用したいモデル名（glm4, llama3, qwen2-vl など）をここに記述
MODEL_NAME = "qwen3-vl:4b"

In [ ]:
def setup_ollama_server(model_name):
    """サーバーを再起動し、指定されたモデルを準備する"""
    print("🔄 サーバーをリセット中...")
    !pkill ollama
    time.sleep(2)

    # サーバーをバックグラウンドで起動
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    time.sleep(5)

    print(f"⏳ モデル '{model_name}' を準備中...")
    subprocess.run(["ollama", "pull", model_name])

    # 接続確認
    try:
        res = requests.get("http://localhost:11434/api/tags")
        if res.status_code == 200:
            models = [m['name'] for m in res.json()['models']]
            print(f"✅ Ollama server is live! 利用可能モデル: {models}")
            return True
    except:
        print("❌ サーバーの起動に失敗しました。")
        return False

# 実行
setup_ollama_server(MODEL_NAME)

In [ ]:
import gradio as gr
import requests
import base64
import nbformat
from pypdf import PdfReader

def extract_text_from_file(file_path):
    ext = file_path.lower()
    try:
        if ext.endswith('.ipynb'):
            with open(file_path, 'r', encoding='utf-8') as f:
                nb = nbformat.read(f, as_version=4)
                return "\n".join([f"[{c.cell_type}]\n{c.source}" for c in nb.cells if c.cell_type in ['code', 'markdown']])
        elif ext.endswith('.pdf'):
            reader = PdfReader(file_path)
            return "\n".join([p.extract_text() for p in reader.pages if p.extract_text()])
        # テキスト系ファイル（HTML, JS, CSS, Py, etc.）
        elif ext.endswith(('.txt', '.py', '.md', '.html', '.js', '.css', '.json', '.ts')):
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
    except Exception as e:
        return f"読込エラー ({file_path}): {str(e)}"
    return None

def chat_with_model(message, history):
    user_text = message["text"]
    user_files = message["files"]
    
    model = MODEL_NAME if 'MODEL_NAME' in globals() else "qwen3-vl:4b"
    url = "http://localhost:11434/api/chat"

    file_contents = []
    images_base64 = []

    # 複数ファイルをスキャン
    for f in user_files:
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
            images_base64.append(encode_image_to_base64(f))
        else:
            content = extract_text_from_file(f)
            if content:
                # どのファイルの内容か明確にするための見出しを付ける
                file_contents.append(f"ファイル名: {f}\n内容:\n{content}\n" + "="*30)

    # 統合プロンプトの作成
    if file_contents:
        all_files_text = "\n\n".join(file_contents)
        full_prompt = (
            f"以下の関連するファイル群を解析してください:\n\n{all_files_text}\n\n"
            f"ユーザーの依頼: {user_text}"
        )
    else:
        full_prompt = user_text

    current_msg = {"role": "user", "content": full_prompt}
    if images_base64:
        current_msg["images"] = images_base64

    # メッセージ履歴の構築
    msgs = []
    for user_h, assistant_h in history:
        msgs.append({"role": "user", "content": user_h})
        msgs.append({"role": "assistant", "content": assistant_h})
    msgs.append(current_msg)

    try:
        response = requests.post(url, json={"model": model, "messages": msgs, "stream": False}, timeout=180)
        return response.json()['message']['content']
    except Exception as e:
        return f"⚠️ エラー: {str(e)}"

# UI起動 (複数選択を許可)
demo = gr.ChatInterface(
    fn=chat_with_model,
    title="📂 Multi-File & Full-Stack AI Explorer",
    multimodal=True,
    textbox=gr.MultimodalTextbox(
        file_types=["image", ".txt", ".pdf", ".ipynb", ".py", ".html", ".js", ".css", ".ts"],
        placeholder="ファイルを複数選択（ドラッグ＆ドロップ）して、質問を入力してください..."
    ),
    theme="soft"
)

demo.launch(share=True, debug=True)

In [ ]:
# def launch_vscode_tunnel():
#     """VS Code Tunnelを起動し、GitHub認証URLを表示する"""
#     import os

#     # ファイルが存在しない場合のみダウンロード
#     if not os.path.exists("code"):
#         print("⏳ VS Code CLI をダウンロード中...")
#         !curl -Lk 'https://code.visualstudio.com/sha/download?build=stable&os=cli-alpine-x64' --output vscode_cli.tar.gz
#         !tar -xf vscode_cli.tar.gz

#     print("\n" + "="*50)
#     print("🚀 VS Code Tunnel を開始します...")
#     print("※ URLが表示されるまで 30秒〜1分ほどかかる場合があります。")
#     print("="*50 + "\n")

#     # 認証情報をリセットして確実に新しいリンクを出す
#     # 以前の `--provider` エラーを避けるため、最小限の引数で実行します
#     !./code tunnel user logout
#     !./code tunnel --accept-server-license-terms

In [ ]:
# !pkill code  # 残っているVS Codeプロセスを強制終了